In [1]:
import sys
import io
import os
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import os, warnings
import re
warnings.filterwarnings(action='ignore')

In [2]:
# 내가 원하는 상품의 리뷰 페이지 여러개 가져와보기

driver = webdriver.Chrome("./chromedriver_90")
url = 'https://www.amazon.com'
driver.get(url)

# 아마존 메인 페이지의 검색 버튼의 요소를 가져온다. by xpath를 이용하는 게 가장 편리하고 정확하다.

sel_search = driver.find_element_by_xpath('//*[@id="twotabsearchtextbox"]')
sel_search_btn = driver.find_element_by_xpath('//*[@id="nav-search-submit-button"]')

print(sel_search.tag_name, sel_search_btn.tag_name)

input input


In [3]:
sel_search.clear()
sel_search.send_keys('anxious people')
sel_search_btn.click()

In [7]:
# 첫번째 제품의 링크 가져와서 페이지로 이동하기

first_link = driver.find_element_by_xpath('//*[@id="search"]/div[1]/div/div[1]/div/span[3]/div[2]/div[1]/div/span/div/div/div[2]/div[2]/div/div[1]/h2/a')
first_link.click()

In [8]:
# 모든 리뷰를 보기 위한 페이지로 이동하기
all_review_link = driver.find_element_by_xpath('//*[@id="reviews-medley-footer"]/div[2]/a')
all_review_link.click()

In [9]:
# 10 페이지의 리뷰 데이터 크롤링해오기

all_names = []
all_scores = []
all_locations = []
all_dates = []
all_helpfuls = []
all_comments = []

for page_number in range(10):
    time.sleep(5)

    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")
    review_table = soup.find("div", id="cm_cr-review_list")
    review_list = review_table.find_all("div", class_="a-section review aok-relative")

    for review in review_list:
        # name
        name = review.find("div", class_="a-profile-content").find("span", class_="a-profile-name").text
        all_names.append(name)

        # score
        score = review.find("span", class_="a-icon-alt").text.split(" ")[0].strip()
        all_scores.append(score)

        # location & date
        location = review.find("div", class_="a-row a-spacing-none").find("span", class_="a-size-base a-color-secondary review-date").text.split("on")[0].strip()
        date = review.find("div", class_="a-row a-spacing-none").find("span", class_="a-size-base a-color-secondary review-date").text.split("on")[-1].strip()
        all_locations.append(location)
        all_dates.append(date)

        # helpful
        try:
            helpful = review.find("span", class_="a-size-base a-color-tertiary cr-vote-text").text.strip()        
        
        except:
            helpful = "0 people found this helpful"

        all_helpfuls.append(helpful)

        # comment
        comment = review.find("div", class_="a-row a-spacing-none").find("span", class_="a-size-base review-text review-text-content").text.strip()
        all_comments.append(comment)

    #1 페이지 크롤링 후 확인
    print("review number: ", len(all_comments))
    print("user: ", all_names[-1], "score:", all_scores[-1])
    print("location: ", all_locations[-1], "date: ", all_dates[-1])
    print("number of people get helped: ", all_helpfuls[-1])
    print("comment: ", all_comments[-1], end="\n\n")

    # 다음 페이지 이동
    next_page_link = driver.find_element_by_xpath('//*[@id="cm_cr-pagination_bar"]/ul/li[2]/a')
    next_page_link.click()

review number:  10
user:  Lynn K. score: 3.0
location:  Reviewed in the United States date:  September 19, 2020
number of people get helped:  35 people found this helpful
comment:  I wanted to like this book and there are some startling and amusing insights, but it was too annoying to read.  With a rambling story line and mostly annoying characters it quickly becomes too tedious.

review number:  20
user:  victoria hill score: 1.0
location:  Reviewed in the United States date:  September 18, 2020
number of people get helped:  19 people found this helpful
comment:  i hated this book so stupid. i found no humor at all.

review number:  30
user:  olmy score: 2.0
location:  Reviewed in the United States date:  October 22, 2020
number of people get helped:  7 people found this helpful
comment:  the beginning of the story was nice, that's why I bought it, but then it goes into the same thing over and over again. You know after the first 20 pages what's going on and then it drags along. You w

In [16]:
print(len(all_names))
print(len(all_scores))
print(len(all_locations))
print(len(all_dates))
print(len(all_helpfuls))
print(len(all_comments))

name_dict= {"name": all_names}
score_dict = {"score": all_scores}
location_dict = {"location": all_locations}
date_dict = {"date": all_dates}
helpful_dict = {"helpful": all_helpfuls}
comment_dict = {"comment": all_comments}

name_df = pd.DataFrame(name_dict)
score_df = pd.DataFrame(score_dict)
location_df = pd.DataFrame(location_dict)
date_df = pd.DataFrame(date_dict)
helpful_df = pd.DataFrame(helpful_dict)
comment_df = pd.DataFrame(comment_dict)

review_df = pd.concat([name_df, score_df, location_df, date_df, helpful_df, comment_df], axis=1)
review_df

100
100
100
100
100
100


,name,score,location,date,helpful,comment
0,Bianca Trevino,5.0,Reviewed in the United States,"September 8, 2020",322 people found this helpful,I'm a little biased but Backman can do no wron...
1,Patti,5.0,Reviewed in the United States,"September 12, 2020",198 people found this helpful,Oh my gosh I loved this book - more than I lov...
2,cocochanel20,5.0,Reviewed in the United States,"September 9, 2020",191 people found this helpful,This book had a complex start and set up but i...
3,Schinnie,2.0,Reviewed in the United States,"September 30, 2020",144 people found this helpful,Oh my gosh is this the same Fredrik Bachman wh...
4,Denice J. Lipscomb,5.0,Reviewed in the United States,"September 14, 2020",132 people found this helpful,I read this book in two days. I had to force m...
...,...,...,...,...,...,...
95,Shud49,2.0,Reviewed in the United States,"October 21, 2020",2 people found this helpful,"Something I rarely, very rarely do. Got 51% th..."
96,Sheila,5.0,Reviewed in the United States,"September 24, 2020",One person found this helpful,So often when I've read all of an author their...
97,Retired in Florida,5.0,Reviewed in the United States,"May 21, 2021",0 people found this helpful,"Bachman has the Scandinavian dry, sardonic hum..."
98,J. Kuntze,5.0,Reviewed in the United States,"November 1, 2020",0 people found this helpful,There are many writers who have something to s...


In [15]:
review_df.to_excel("book anxious people review data.xlsx")
review_df.to_csv("book anxious people review data.csv")